In [1]:
import os
import re
import unicodecsv
import django
django.setup()
from sefaria.model import *
from collections import defaultdict, namedtuple
from sefaria.datatype import jagged_array
from data_utilities.util import getGematria

In [2]:
RefTracker = namedtuple('RefTracker', ['ref', 'indices'])

In [3]:
def collect_sections(rows):
    ref_list, prev_ref = [], None
    section_map = defaultdict(list)
    for row in rows:
        cur_ref = Ref(row[u'EY current paragraph'])
        cur_ref_normal = cur_ref.normal()
        
        if cur_ref_normal != prev_ref:
            ref_list.append(RefTracker(cur_ref_normal, [i-1 for i in cur_ref.sections]))
            prev_ref = cur_ref_normal
            
        section_map[cur_ref_normal].append(row[u'EY comment'])
    
    ref_list.sort(key=lambda x: x.indices)
    ja = jagged_array.JaggedArray()
    for ref in ref_list:
        ja.set_element(ref.indices, section_map[ref.ref], pad=[])
    
    return ja.array()

In [4]:
def collect_sections_genesis(rows):
    def zero_index_gematria(gem): return getGematria(gem) - 1
    
    parsha_reg = re.compile(u'\u05e4\u05e8\u05e9\u05d4 ([\u05d0-\u05ea]{1,2})')
    current_section = []
    indices, new_indices = [0, 0], [0, 0]
    ja = jagged_array.JaggedArray()
    
    for row in rows:    
        if row[0]:
            new_indices[0] = zero_index_gematria(parsha_reg.search(row[0]).group(1))
        if row[1]:
            new_indices[1] = zero_index_gematria(row[1])
            
        if new_indices != indices:
            ja.set_element(indices, current_section, pad=[])
            indices = new_indices[:]
            current_section = []
        current_section.append(row[2])
    
    return ja.array()

Etz Yosef on Midrash [Book]

Midrash Rabbah, with Etz Yosef, Warsaw, 1867


In [5]:
from sources.functions import post_index, post_text, add_term, add_category

In [26]:
server = 'https://www.sefaria.org'

In [27]:
add_term("Etz Yosef", u'עץ יוסף', server=server)

In [28]:
add_category("Etz Yosef", ["Midrash", "Aggadic Midrash", "Midrash Rabbah", "Commentary", "Etz Yosef"], server=server)

In [24]:
def create_index(book_name):
    node = JaggedArrayNode()
    he_name = Ref(book_name).he_book()
    en_title = "Etz Yosef on {}".format(book_name)
    he_title = u"עץ יוסף על {}".format(he_name)
    node.add_primary_titles(en_title, he_title)
    node.add_structure(["Chapter", "Paragraph", "Comment"])
    node.toc_zoom =2
    return {
        u'title': en_title,
        u'categories': ["Midrash", "Aggadic Midrash", "Midrash Rabbah", "Commentary", "Etz Yosef"],
        u'dependence': u'Commentary',
        u'collective_title': u'Etz Yosef',
        u'schema': node.serialize(),
        u'base_text_titles': [book_name],
        u"base_text_mapping": u"many_to_one"
    }

In [11]:
def create_version(text_array):
    return {
        u'versionTitle': u'Midrash Rabbah, with Etz Yosef, Warsaw, 1867',
        u'versionSource': u'http://merhav.nli.org.il/primo-explore/fulldisplay?docid=NNL_ALEPH001987082&context=L&vid=NLI&search_scope=Local&tab=default_tab&lang=iw_IL',
        u'language': u'he',
        u'text': text_array
    }

In [12]:
def upload_text(filename, base_en, dest):
    with open(filename) as fp:
        if filename == u'Etz_Yosef_-_Genesis_-_tags_corrected.csv':
            text_array = collect_sections_genesis(unicodecsv.reader(fp))
        else:
            text_array = collect_sections(unicodecsv.DictReader(fp))
    version = create_version(text_array)
    index = create_index(base_en)
    post_index(index, dest)
    post_text(index['title'], version, index_count="on", server=dest)

In [31]:
upload_text(u'Etz_Yosef_-_Genesis_-_tags_corrected.csv', u'Bereishit Rabbah', server)
upload_text(u'Etz_Yoseph_-_Exodus.csv', u'Shemot Rabbah', server)
upload_text(u'Etz_Yoseph_-_Leviticus.csv', u'Vayikra Rabbah', server)
upload_text(u'Etz_Yoseph_-_Bamidbar.csv', u'Bamidbar Rabbah', server)
upload_text(u'Etz_Yoseph_-_Deuteronomy.csv', u'Devarim Rabbah', server)